In [ ]:
import numpy as np
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from math import log10

In [ ]:
sf = gpd.read_file('./data_for_paper_final_version/Grid06.shp')
bitcoin = pd.read_csv('./data_for_paper_final_version/Aggregated harsh rate by unique address(201806-201905).csv')
powerdata = pd.read_csv('./data_for_paper_final_version/global_power_plant_database.csv')
data = powerdata.groupby(['latitude','longitude'])

In [ ]:
months = [i for i in range(1806, 1813)] + [i for i in range(1901, 1906)]
for month in months:
    month_data = bitcoin[bitcoin['Month'] == month+200000]
    month_hashrate = []
    for index, row in sf.iterrows():
        hash_rate = 0
        for i, each in month_data.iterrows():
            if(row.get('geometry').contains(Point(each.get('longitude'),each.get('latitude')))):
                hash_rate += each.get('Avg Hashrate')
        month_hashrate.append(hash_rate)
    sf['Hrate'+str(month)] = month_hashrate

In [ ]:
hash_rate_all = sf['Hrate1806'].copy(deep=True)
for month in months:
    hash_rate_all += sf['Hrate'+str(month)]

sf['HrateAvg'] = hash_rate_all/12
all_hash_rate = sum(sf['HrateAvg'])
sf['HrateShare'] = sf['HrateAvg'] / all_hash_rate
sf['HrateLG'] = [log10(i+1) for i in sf['HrateAvg']]
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x

sf['HashZS'] = ZscoreNormalization(sf['HrateLG'])

In [ ]:
renewable_data = ['Geothermal', 'Hydro', 'Solar', 'Wind']
fossil_data = ['Oil', 'Gas', 'Petcoke', 'Coal']

renewable = pd.DataFrame(columns=['latitude','longitude', 'capacity'])
fossil = pd.DataFrame(columns=['latitude','longitude', 'capacity'])
all_entypes = pd.DataFrame(columns=['latitude','longitude', 'capacity'])
wind = pd.DataFrame(columns=['latitude','longitude', 'capacity'])
hydro = pd.DataFrame(columns=['latitude','longitude', 'capacity'])
solar = pd.DataFrame(columns=['latitude','longitude', 'capacity'])
geothermal = pd.DataFrame(columns=['latitude','longitude', 'capacity'])

In [ ]:
for index, group in data:
    if (len(group) == 1):
        datalist = [{'latitude': group['latitude'].values[0], 'longitude': group['longitude'].values[0],
                     'capacity': group['capacity_mw'].values[0]}]
        if group['primary_fuel'].values[0] in renewable_data:
            renewable = renewable.append(datalist, ignore_index=True)
            if group['primary_fuel'].values[0] == 'Wind':
                wind = wind.append(datalist, ignore_index=True)
            elif group['primary_fuel'].values[0] == 'Hydro':
                hydro = hydro.append(datalist, ignore_index=True)
            elif group['primary_fuel'].values[0] == 'Solar':
                solar = solar.append(datalist, ignore_index=True)
            elif group['primary_fuel'].values[0] == 'Geothermal':
                geothermal = geothermal.append(datalist, ignore_index=True)
            else:
                print('error: ', index)
        elif group['primary_fuel'].values[0] in fossil_data:
            fossil = fossil.append(datalist, ignore_index=True)
        all_entypes = all_entypes.append(datalist, ignore_index=True)
    else:
        renewable_cap = 0
        fossil_cap = 0
        all_entypes_cap = 0
        wind_cap = 0
        hydro_cap = 0
        solar_cap = 0
        geothermal_cap = 0
        for i, row in group.iterrows():
            if row['primary_fuel'] in renewable_data:
                renewable_cap += row['capacity_mw']
                if row['primary_fuel'] == 'Wind':
                    wind_cap += row['capacity_mw']
                elif row['primary_fuel'] == 'Hydro':
                    hydro_cap += row['capacity_mw']
                elif row['primary_fuel'] == 'Solar':
                    solar_cap += row['capacity_mw']
                elif row['primary_fuel'] == 'Geothermal':
                    geothermal_cap += row['capacity_mw']
                else:
                    print('error: ', index)
            if row['primary_fuel'] in fossil_data:
                fossil_cap += row['capacity_mw']
            all_entypes_cap += row['capacity_mw']
            
        if renewable_cap != 0:
            datalist = [{'latitude': group['latitude'].values[0], 'longitude': group['longitude'].values[0],
                         'capacity': renewable_cap}]
            renewable = renewable.append(datalist, ignore_index=True)
            if wind_cap != 0:
                datalist = [{'latitude': group['latitude'].values[0], 'longitude': group['longitude'].values[0],
                         'capacity': wind_cap}]
                wind = wind.append(datalist, ignore_index=True)
            if hydro_cap != 0:
                datalist = [{'latitude': group['latitude'].values[0], 'longitude': group['longitude'].values[0],
                         'capacity': hydro_cap}]
                hydro = hydro.append(datalist, ignore_index=True)
            if solar_cap != 0:
                datalist = [{'latitude': group['latitude'].values[0], 'longitude': group['longitude'].values[0],
                         'capacity': solar_cap}]
                solar = solar.append(datalist, ignore_index=True)
            if geothermal_cap != 0:
                datalist = [{'latitude': group['latitude'].values[0], 'longitude': group['longitude'].values[0],
                         'capacity': geothermal_cap}]
                geothermal = geothermal.append(datalist, ignore_index=True)

        if fossil_cap != 0:
            datalist = [{'latitude': group['latitude'].values[0], 'longitude': group['longitude'].values[0],
                     'capacity': fossil_cap}]
            fossil = fossil.append(datalist, ignore_index=True)
        
        datalist = [{'latitude': group['latitude'].values[0], 'longitude': group['longitude'].values[0],
                     'capacity': all_entypes_cap}]
        all_entypes = all_entypes.append(datalist, ignore_index=True)


In [ ]:
merge_data = pd.merge(all_entypes,fossil,how='outer',on=['latitude','longitude'])
merge_data = merge_data.rename(columns={'capacity_x':'all_entypes', 'capacity_y':'fossil'})

merge_data = pd.merge(merge_data,renewable,how='outer',on=['latitude','longitude'])
merge_data = merge_data.rename(columns={'capacity':'renewable'})

merge_data = pd.merge(merge_data,hydro,how='outer',on=['latitude','longitude'])
merge_data =merge_data.rename(columns={'capacity':'hydro'})

merge_data = pd.merge(merge_data,solar,how='outer',on=['latitude','longitude'])
merge_data =merge_data.rename(columns={'capacity':'solar'})

merge_data = pd.merge(merge_data,geothermal,how='outer',on=['latitude','longitude'])
merge_data =merge_data.rename(columns={'capacity':'geothermal'})
                              
merge_data = pd.merge(merge_data,wind,how='outer',on=['latitude','longitude'])
merge_data =merge_data.rename(columns={'capacity':'wind'})

merged_data = merge_data.fillna(0)

In [ ]:
all_entypes = []
fossil = []
renewable = []
wind = []
hydro = []
solar = []
geothermal = []
drop_count = 0

for index, value in sf.iterrows():
    all_entypes_value = 0
    fossil_value = 0
    renewable_value = 0
    wind_value = 0
    hydro_value = 0
    solar_value = 0
    geothermal_value = 0
    for i, row in merged_data.iterrows():
        if (value.get('geometry').contains(Point(row.get('longitude'), row.get('latitude')))):
            all_entypes_value += row.get('all_entypes')
            fossil_value += row.get('fossil')
            renewable_value += row.get('renewable')
            wind_value += row.get('wind')
            hydro_value += row.get('hydro')
            solar_value += row.get('solar')
            geothermal_value += row.get('geothermal')
            powers = merged_data.drop(index=[i])
            drop_count += 1
    all_entypes.append(all_entypes_value)
    fossil.append(fossil_value)
    renewable.append(renewable_value)
    wind.append(wind_value)
    hydro.append(hydro_value)
    solar.append(solar_value)
    geothermal.append(geothermal_value)

In [ ]:
sf['Cap.All'] = all_entypes
sf['Cap.Fos'] = fossil
sf['Cap.Ren'] = renewable
sf['Cap.Win'] = wind
sf['Cap.Hyd'] = hydro
sf['Cap.Sol'] = solar
sf['Cap.Geo'] = geothermal

sf['Cap.Fos.LG'] = [log10(i+1) for i in sf['Cap.Fos']]
sf['Cap.Ren.LG'] = [log10(i+1) for i in sf['Cap.Ren']]
sf['Cap.All.LG'] = [log10(i+1) for i in sf['Cap.All']]

sf['Cap.Fos.ZS'] = ZscoreNormalization(sf['Cap.Fos.LG'])
sf['Cap.Ren.ZS'] = ZscoreNormalization(sf['Cap.Ren.LG'])
sf['Cap.All.ZS'] = ZscoreNormalization(sf['Cap.All.LG'])

In [ ]:
sf.to_file('./data_for_paper_final_version/Grids with hash rate and energy(201806-201905).shp')